# **Construindo uma Base de Dados Macroeconômica**
---

**Avisos**:

- Este código realiza coleta de dados online
- Erros de coleta de dados podem acontecer
- Erros costumam ser corrigidos ao tentar novamente
- Se o erro persistir, verifique o funcionamento na fonte de dados

# **Bibliotecas**
---



In [ ]:
# Importa bibliotecas
import pandas as pd
import numpy as np
import os, time
from datetime import datetime, timedelta

# **Funções**
---

In [ ]:
# Retenta ler um CSV se falhar download
def ler_csv(*args, **kwargs):
  max_tentativas = 5
  intervalo = 2
  tentativas = 0
  while tentativas < max_tentativas:
      try:
          df = pd.read_csv(*args, **kwargs)
          return df
      except Exception as e:
          tentativas += 1
          print(f"Tentativa {tentativas} falhou: {e}")
          time.sleep(intervalo)
  print(f"Falha após {max_tentativas} tentativas.")
  return None

In [ ]:
# Separa intervalo de datas em janelas de 10 anos para coleta de dados em blocos
# na API do BCB/SGS
def split_date_range(start_date_str, end_date_str, interval_years=5):
  start_date = datetime.strptime(start_date_str, "%d/%m/%Y")
  end_date = datetime.strptime(end_date_str, "%d/%m/%Y")

  result = []
  current_start = start_date

  while current_start < end_date:
    try:
      current_end = current_start.replace(year=current_start.year + interval_years)
    except ValueError:
      current_end = current_start + timedelta(days=365 * interval_years)

    if current_end > end_date:
      current_end = end_date

    result.append((
      current_start.strftime("%d/%m/%Y"),
      current_end.strftime("%d/%m/%Y")
    ))
    current_start = current_end

  return result

# Coleta dados da API do Banco Central (SGS)
def coleta_bcb_sgs(codigo, nome, freq, data_inicio = "01/01/2000", data_fim = (pd.to_datetime("today") + pd.offsets.DateOffset(months = 36)).strftime("%d/%m/%Y")):

  if freq == "Diária":
    datas_inicio = split_date_range(data_inicio, data_fim)
  else:
    datas_inicio = [(data_inicio, data_fim)]

  try:
    print(f"Coletando a série {codigo} ({nome})")
    resposta = []
    for d in datas_inicio:
      url = f"https://api.bcb.gov.br/dados/serie/bcdata.sgs.{codigo}/dados?formato=csv&dataInicial={d[0]}&dataFinal={d[1]}"
      resposta.append(ler_csv(filepath_or_buffer = url, sep = ";", decimal = ","))
    resposta = pd.concat(resposta)
  except:
    raise Exception(f"Falha na coleta da série {codigo} ({nome})")
  else:
    return (
        resposta
        .rename(columns = {"valor": nome})
        .assign(data = lambda x: pd.to_datetime(x.data, format = "%d/%m/%Y"))
        .set_index("data")
    )

In [ ]:
# Coleta dados da API do Banco Central (ODATA)
def coleta_bcb_odata(codigo, nome):

  url = codigo

  try:
    print(f"Coletando a série {codigo} ({nome})")
    resposta = ler_csv(
        filepath_or_buffer = url,
        sep = ",", decimal = ",",
        converters = {"Data": lambda x: pd.to_datetime(x)}
        )
  except:
    raise Exception(f"Falha na coleta da série {codigo} ({nome})")
  else:
    return resposta.rename(columns = {"Mediana": nome})

In [ ]:
# Coleta dados da API do IPEA (IPEADATA)
def coleta_ipeadata(codigo, nome):

  url = f"http://www.ipeadata.gov.br/api/odata4/ValoresSerie(SERCODIGO='{codigo}')"
  try:
    print(f"Coletando a série {codigo} ({nome})")
    resposta = pd.read_json(url)
  except:
    raise Exception(f"Falha na coleta da série {codigo} ({nome})")
  else:
    df = (
        pd.DataFrame.from_records(resposta["value"])
        .rename(columns = {"VALVALOR": nome, "VALDATA": "data"})
        .filter(["data", nome])
      )
  if df.empty:
    raise Exception(f"Falha na coleta da série {codigo} ({nome})")
  else:
    return df

In [ ]:
# Coleta dados da API do IBGE (SIDRA)
def coleta_ibge_sidra(codigo, nome):

  url = f"{codigo}?formato=json"
  try:
    print(f"Coletando a série {codigo} ({nome})")
    resposta = pd.read_json(url)
  except:
    raise Exception(f"Falha na coleta da série {codigo} ({nome})")
  else:
    df = (
        resposta
        .rename(columns = {"D3C": "data", "V": nome})
        .filter(["data", nome])
      )
    df = df[-df[nome].isin(["Valor", "...", "-"])]
    df[nome] = pd.to_numeric(df[nome])
    return df

In [ ]:
# Coleta dados da API do FRED
def coleta_fred(codigo, nome):

  url = f"https://fred.stlouisfed.org/graph/fredgraph.csv?id={codigo}"

  try:
    print(f"Coletando a série {codigo} ({nome})")
    resposta = ler_csv(
        filepath_or_buffer = url,
        converters = {"DATE": lambda x: pd.to_datetime(x)}
        )
  except:
    raise Exception(f"Falha na coleta da série {codigo} ({nome})")
  else:
    return resposta.rename(columns = {"DATE": "data", codigo: nome})

In [ ]:
# Coleta dados via link da IFI
def coleta_ifi(codigo, nome):

  try:
    print(f"Coletando a série {codigo} ({nome})")
    resposta = pd.read_excel(
        io = codigo,
        sheet_name = "Hiato do Produto",
        names = ["data", "lim_inf", nome, "lim_sup"],
        skiprows = 2
        )
  except:
    raise Exception(f"Falha na coleta da série {codigo} ({nome})")
  else:
    return resposta

# **Coleta de dados**
---

In [ ]:
# Planilha de metadados
df_metadados = pd.read_excel(
    io = "https://docs.google.com/spreadsheets/d/1x8Ugm7jVO7XeNoxiaFPTPm1mfVc3JUNvvVqVjCioYmE/export?format=xlsx",
    sheet_name = "Metadados"
    )
df_metadados.head()

,Tema,Variável,Descrição,Unidade,Região,Frequência,Fonte,Forma de Coleta,Input de Coleta,Identificador,Transformação,Tipo
0,Atividade Econômica,"Fabricação de celulose, papel e produtos de papel","Variação mês/mês imediatamente anterior, com a...",m/m %,Brasil,Mensal,IBGE/SIDRA,API,https://apisidra.ibge.gov.br/values/t/8888/n1/...,prod_ind_papel,1,Independente
1,Atividade Econômica,Fabricação de cimento,Variação mês/mês ano anterior,m/m-12 %,Brasil,Mensal,IBGE/SIDRA,API,https://apisidra.ibge.gov.br/values/t/8885/n1/...,prod_ind_cimento,1,Independente
2,Atividade Econômica,Fabricação de produtos de borracha e de materi...,"Variação mês/mês imediatamente anterior, com a...",m/m %,Brasil,Mensal,IBGE/SIDRA,API,https://apisidra.ibge.gov.br/values/t/8888/n1/...,prod_ind_borracha,1,Independente
3,Atividade Econômica,Hiato do Produto,"IFI, Função de Produção",%,Brasil,Trimestral,IFI,Link,https://www12.senado.leg.br/ifi/dados/arquivos...,hiato_produto,1,Independente
4,Atividade Econômica,Imposto sobre a circulação de mercadorias (ICMS),Mil,R$,Brasil,Mensal,IPEADATA,API,CONFAZ12_ICMSN12,icms,5,Independente


## BCB/SGS

In [ ]:
# Coleta dados do BCB/SGS
input_bcb_sgs = (
    df_metadados
    .query("Fonte == 'BCB/SGS' and `Forma de Coleta` == 'API'")
    .reset_index(drop = True)
)

df_bruto_bcb_sgs = {"Diária": [], "Mensal": [], "Trimestral": [], "Anual": []}

for serie in input_bcb_sgs.index:
  ser = input_bcb_sgs.iloc[serie]
  df_temp = coleta_bcb_sgs(
      codigo = ser["Input de Coleta"],
      nome = ser["Identificador"],
      freq = ser["Frequência"]
      )
  df_bruto_bcb_sgs[ser["Frequência"]].append(df_temp)

Coletando a série 24363 (ibc_br)
Coletando a série 1338 (ind_transformacao_cni)
Coletando a série 24348 (n_hr_trab_ind)
Coletando a série 4192 (pib_acum12m)
Coletando a série 4380 (pib_mensal)
Coletando a série 7357 (prod_aco)
Coletando a série 1389 (prod_oleo_bruto)
Coletando a série 1373 (prod_veic_total)
Coletando a série 28553 (trafego_veic_pesados)
Coletando a série 1344 (uci_geral_fgv)
Coletando a série 24351 (uci_ind_cni)
Coletando a série 24352 (uci_ind_fgv)
Coletando a série 7389 (vendas_veic_concessionarias)
Coletando a série 28549 (vendas_supermercados)
Coletando a série 1378 (venda_veic_total)
Coletando a série 3698 (cambio)
Coletando a série 432 (selic)
Coletando a série 1402 (cons_energia_comercial)
Coletando a série 1404 (cons_energia_ind)
Coletando a série 1403 (cons_energia_residencial)
Coletando a série 1406 (cons_energia_total)
Coletando a série 1393 (cons_gasolina)
Coletando a série 1396 (cons_diesel)
Coletando a série 29038 (endividamento_familias_exhabit)
Coletand

## BCB/ODATA

In [ ]:
# Coleta dados do BCB/ODATA
input_bcb_odata = (
    df_metadados
    .query("Fonte == 'BCB/ODATA' and `Forma de Coleta` == 'API'")
    .reset_index(drop = True)
)

df_bruto_bcb_odata = []

for serie in input_bcb_odata.index:
  ser = input_bcb_odata.iloc[serie]
  df_temp = coleta_bcb_odata(
      codigo = ser["Input de Coleta"],
      nome = ser["Identificador"]
      )
  df_bruto_bcb_odata.append(df_temp)

Coletando a série https://olinda.bcb.gov.br/olinda/servico/Expectativas/versao/v1/odata/ExpectativasMercadoTop5Mensais?$filter=Indicador%20eq%20'IPCA'%20and%20Data%20ge%20'2000-01-01'%20and%20tipoCalculo%20eq%20'C'&$format=text/csv (expec_ipca_top5_curto_prazo)
Coletando a série https://olinda.bcb.gov.br/olinda/servico/Expectativas/versao/v1/odata/ExpectativasMercadoTop5Mensais?$filter=Indicador%20eq%20'IPCA'%20and%20Data%20ge%20'2000-01-01'%20and%20tipoCalculo%20eq%20'M'&$format=text/csv (expec_ipca_top5_medio_prazo)
Coletando a série https://olinda.bcb.gov.br/olinda/servico/Expectativas/versao/v1/odata/ExpectativasMercadoAnuais?$filter=Indicador%20eq%20'Selic'%20and%20baseCalculo%20eq%200&$orderby=Data%20desc&$format=text/csv (expec_selic)
Coletando a série https://olinda.bcb.gov.br/olinda/servico/Expectativas/versao/v1/odata/ExpectativaMercadoMensais?$filter=Indicador%20eq%20'C%C3%A2mbio'%20and%20baseCalculo%20eq%200&$orderby=Data%20asc&$format=text/csv (expec_cambio)
Coletando a sé

## IPEADATA

In [ ]:
# Coleta dados do IPEADATA
input_ipeadata = (
    df_metadados
    .query("Fonte == 'IPEADATA' and `Forma de Coleta` == 'API'")
    .reset_index(drop = True)
)

df_bruto_ipeadata = {"Diária": [], "Mensal": []}

for serie in input_ipeadata.index:
  ser = input_ipeadata.iloc[serie]
  df_temp = coleta_ipeadata(
      codigo = ser["Input de Coleta"],
      nome = ser["Identificador"]
      )
  df_bruto_ipeadata[ser["Frequência"]].append(df_temp)

Coletando a série CONFAZ12_ICMSN12 (icms)
Coletando a série CNI12_HTRAB12 (horas_trab_ind)
Coletando a série ABPO12_PAPEL12 (papelao_ondulado)
Coletando a série GAC12_CAPAPEL12 (cons_papel)
Coletando a série ANP12_CGASOL12 (cons_gasolina_anp)
Coletando a série FENABRAVE12_VENDVETOT12 (emplacamento_veic)
Coletando a série ELETRO12_CEET12 (cons_energia_eletrica)
Coletando a série SRF12_IR12 (ir)
Coletando a série CAGED12_SALDO12 (saldo_caged_antigo)
Coletando a série CAGED12_SALDON12 (saldo_caged_novo)
Coletando a série JPM366_EMBI366 (embi_br)
Coletando a série ANBIMA12_IBVSP12 (ibov)
Coletando a série IFS12_PETROLEUM12 (cotacao_petroleo_fmi)
Coletando a série IGP366_IPCS366 (ipc_s)
Coletando a série FUNCEX12_XVAGP2N12 (exp_agro)


## IBGE/SIDRA

In [ ]:
# Coleta dados do IBGE/SIDRA
input_sidra = (
    df_metadados
    .query("Fonte == 'IBGE/SIDRA' and `Forma de Coleta` == 'API'")
    .reset_index(drop = True)
)

df_bruto_ibge_sidra = {"Mensal": [], "Trimestral": []}

for serie in input_sidra.index:
  ser = input_sidra.iloc[serie]
  df_temp = coleta_ibge_sidra(
      codigo = ser["Input de Coleta"],
      nome = ser["Identificador"]
      )
  df_bruto_ibge_sidra[ser["Frequência"]].append(df_temp)

Coletando a série https://apisidra.ibge.gov.br/values/t/8888/n1/all/v/12607/p/all/c544/129324/d/v12607%205 (prod_ind_papel)
Coletando a série https://apisidra.ibge.gov.br/values/t/8885/n1/all/v/11602/p/all/c542/129233/d/v11602%201 (prod_ind_cimento)
Coletando a série https://apisidra.ibge.gov.br/values/t/8888/n1/all/v/12607/p/all/c544/129331/d/v12607%205 (prod_ind_borracha)
Coletando a série https://apisidra.ibge.gov.br/values/t/5906/n1/all/v/11623/p/all/c11046/56726/d/v11623%201 (pms_volume)
Coletando a série https://apisidra.ibge.gov.br/values/t/8880/n1/all/v/11708/p/all/c11046/56734/d/v11708%201 (pmc_volume)
Coletando a série https://apisidra.ibge.gov.br/values/t/8881/n1/all/v/11708/p/all/c11046/56736/d/v11708%201 (pmc_volume_ampliado)
Coletando a série https://apisidra.ibge.gov.br/values/t/8888/n1/all/v/12607/p/all/c544/129316/d/v12607%205 (prod_ind_transformacao)
Coletando a série https://apisidra.ibge.gov.br/values/t/8887/n1/all/v/11601/p/all/c543/129300/d/v11601%201 (prod_ind_be

## FRED

In [ ]:
# Coleta dados do FRED
input_fred = (
    df_metadados
    .query("Fonte == 'FRED' and `Forma de Coleta` == 'API'")
    .reset_index(drop = True)
)

df_bruto_fred = {"Diária": [], "Mensal": [], "Trimestral": []}

for serie in input_fred.index:
  ser = input_fred.iloc[serie]
  df_temp = coleta_fred(
      codigo = ser["Input de Coleta"],
      nome = ser["Identificador"]
      )
  df_bruto_fred[ser["Frequência"]].append(df_temp)

Coletando a série GS3M (us_gov_sec_3m)
Coletando a série USEPUINDXD (epu_us)
Coletando a série FEDFUNDS (fed_funds)
Coletando a série GS1 (us_gov_sec_1y)
Coletando a série GS10 (us_gov_sec_10y)
Coletando a série GS2 (us_gov_sec_2y)
Coletando a série GS5 (us_gov_sec_5y)
Coletando a série USRECD (nber_us_daily)
Coletando a série NC000334Q (us_gdp)
Coletando a série ICSA (initial_claims)


## IFI

In [ ]:
# Coleta dados do IFI
input_ifi = (
    df_metadados
    .query("Fonte == 'IFI'")
    .reset_index(drop = True)
)

df_bruto_ifi = coleta_ifi(input_ifi["Input de Coleta"][0], input_ifi["Identificador"][0])

Coletando a série https://www12.senado.leg.br/ifi/dados/arquivos/estimativas-do-hiato-do-produto-ifi (hiato_produto)


# **Tratamento de dados**
---

## BCB/SGS

In [ ]:
# Cruza dados do BCB/SGS
df_tratado_bcb_sgs = df_bruto_bcb_sgs.copy()

for f in df_tratado_bcb_sgs.items():
  df_temp = f[1][0]
  for df in f[1][1:]:
    df_temp = df_temp.join(other = df, how = "outer")
  df_tratado_bcb_sgs[f[0]] = df_temp

df_tratado_bcb_sgs["Mensal"]

,ibc_br,ind_transformacao_cni,n_hr_trab_ind,pib_acum12m,pib_mensal,prod_aco,prod_oleo_bruto,prod_veic_total,trafego_veic_pesados,uci_ind_cni,...,ipca_nucleo_p55,ipca_nucleo_ex0,ipca_nucleo_ex1,ipca_nucleo_ex2,ipca_nucleo_ex3,ipca_servicos,bc_saldo,idp,mov_cambio_contr,tc_saldo
data,,,,,,,,,,,,,,,,,,,,,
2000-01-01,NaN,NaN,NaN,602406.0,92576.6,120.48,1155.0,92704.0,103.80,NaN,...,0.34,0.56,0.63,0.42,0.33,0.69,-667.2,3014.9,1056.463000,-1049.0
2000-02-01,NaN,NaN,NaN,608291.0,91770.4,108.50,1121.0,126595.0,100.74,NaN,...,0.13,0.45,0.32,0.17,0.21,0.38,-473.3,1691.2,144.661000,-1306.8
2000-03-01,NaN,NaN,NaN,612496.0,92579.9,124.50,1189.0,130730.0,101.64,NaN,...,0.10,0.05,0.16,-0.11,-0.03,-0.11,-780.9,2241.8,-614.645914,-2049.6
2000-04-01,NaN,NaN,NaN,615430.0,91376.2,116.50,1161.0,129116.0,103.43,NaN,...,0.18,0.87,0.62,0.16,0.22,1.47,-540.9,1386.5,-341.510617,-3122.3
2000-05-01,NaN,NaN,NaN,620436.0,98727.0,124.00,1159.0,144738.0,99.32,NaN,...,0.01,0.20,0.19,0.12,0.16,-0.07,-456.7,1669.5,571.662406,-1767.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-01,148.86,103.0,94.5,2240287.0,953614.0,130.04,3318.0,166732.0,167.28,79.2,...,0.45,0.36,0.42,0.38,0.37,0.40,2982.4,3023.0,-712.599847,-2273.8
2024-06-01,NaN,NaN,NaN,2232692.0,949831.7,144.62,NaN,211035.0,178.63,NaN,...,0.30,0.09,0.16,0.27,0.27,0.04,1900.0,6269.5,5603.491209,-4029.0
2024-07-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Agrega dados de frequência diária para mensal por média ou início de mês
df_tratado_bcb_sgs["Mensal"] = df_tratado_bcb_sgs["Mensal"].join(
    other = (
        df_tratado_bcb_sgs["Diária"]
        .filter(input_bcb_sgs.query("Identificador != 'selic'")["Identificador"].to_list())
        .resample("MS")
        .mean()
        .join(
            other = (
                df_tratado_bcb_sgs["Diária"]
                .filter(["selic"])
                .reset_index()
                .assign(data = lambda x: x.data.dt.to_period("M").dt.to_timestamp())
                .groupby("data")
                .head(1)
                .set_index("data")
            ),
            how = "outer"
        )
        .query("index >= '2000-01-01'")
    ),
    how = "outer"
)
df_tratado_bcb_sgs["Mensal"]

,ibc_br,ind_transformacao_cni,n_hr_trab_ind,pib_acum12m,pib_mensal,prod_aco,prod_oleo_bruto,prod_veic_total,trafego_veic_pesados,uci_ind_cni,...,bc_saldo,idp,mov_cambio_contr,tc_saldo,op_titulos_federais,papel_moeda,cambio_brl_eur,cambio_brl_gbp,cambio_brl_jpy,selic
data,,,,,,,,,,,,,,,,,,,,,
2000-01-01,NaN,NaN,NaN,602406.0,92576.6,120.48,1155.0,92704.0,103.80,NaN,...,-667.2,3014.9,1056.463000,-1049.0,-6.859782e+06,2.640321e+07,1.831620,2.962041,0.017131,19.00
2000-02-01,NaN,NaN,NaN,608291.0,91770.4,108.50,1121.0,126595.0,100.74,NaN,...,-473.3,1691.2,144.661000,-1306.8,1.234023e+06,2.428453e+07,1.750825,2.845847,0.016220,19.00
2000-03-01,NaN,NaN,NaN,612496.0,92579.9,124.50,1189.0,130730.0,101.64,NaN,...,-780.9,2241.8,-614.645914,-2049.6,6.463143e+05,2.363977e+07,1.684744,2.759518,0.016392,19.00
2000-04-01,NaN,NaN,NaN,615430.0,91376.2,116.50,1161.0,129116.0,103.43,NaN,...,-540.9,1386.5,-341.510617,-3122.3,1.165958e+06,2.357564e+07,1.674534,2.800036,0.016735,18.50
2000-05-01,NaN,NaN,NaN,620436.0,98727.0,124.00,1159.0,144738.0,99.32,NaN,...,-456.7,1669.5,571.662406,-1767.2,2.594206e+06,2.351213e+07,1.662632,2.758030,0.016902,18.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-01,148.86,103.0,94.5,2240287.0,953614.0,130.04,3318.0,166732.0,167.28,79.2,...,2982.4,3023.0,-712.599847,-2273.8,-5.218370e+07,3.377881e+08,5.553352,6.490571,0.032949,10.75
2024-06-01,NaN,NaN,NaN,2232692.0,949831.7,144.62,NaN,211035.0,178.63,NaN,...,1900.0,6269.5,5603.491209,-4029.0,-1.073908e+08,3.458602e+08,5.800010,6.851945,0.034138,10.50
2024-07-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-9.547480e+07,3.486373e+08,6.011000,7.129339,0.035171,10.50


## BCB/ODATA

In [ ]:
# Filtra expectativas curto prazo ~1 mês à frente e agrega pela média
df_tratado_bcb_odata_ipca_cp = (
    df_bruto_bcb_odata[0]
    .assign(
        DataReferencia = lambda x: pd.to_datetime(x.DataReferencia, format = "%m/%Y"),
        horizonte = lambda x: ((x.DataReferencia - x.Data) / np.timedelta64(30, "D")).astype(int),
        data = lambda x: x.Data.dt.to_period("M").dt.to_timestamp()
        )
    .query("horizonte == 1")
    .groupby(["data"], as_index = False)["expec_ipca_top5_curto_prazo"]
    .mean()
)
df_tratado_bcb_odata_ipca_cp

,data,expec_ipca_top5_curto_prazo
0,2001-11-01,0.452778
1,2001-12-01,0.336842
2,2002-01-01,0.273636
3,2002-02-01,0.354211
4,2002-03-01,0.353500
...,...,...
269,2024-04-01,0.162133
270,2024-05-01,0.134762
271,2024-06-01,0.062500
272,2024-07-01,0.184091


In [ ]:
# Filtra expectativas médio prazo ~6 mês à frente e agrega pela média
df_tratado_bcb_odata_ipca_mp = (
    df_bruto_bcb_odata[1]
    .assign(
        DataReferencia = lambda x: pd.to_datetime(x.DataReferencia, format = "%m/%Y"),
        horizonte = lambda x: ((x.DataReferencia - x.Data) / np.timedelta64(30, "D")).astype(int),
        data = lambda x: x.Data.dt.to_period("M").dt.to_timestamp()
        )
    .query("horizonte == 6")
    .groupby(["data"], as_index = False)["expec_ipca_top5_medio_prazo"]
    .mean()
)
df_tratado_bcb_odata_ipca_mp

,data,expec_ipca_top5_medio_prazo
0,2001-11-01,0.605556
1,2001-12-01,0.854000
2,2002-01-01,0.514762
3,2002-02-01,0.234211
4,2002-03-01,0.350000
...,...,...
269,2024-04-01,0.276171
270,2024-05-01,0.500638
271,2024-06-01,0.454568
272,2024-07-01,0.501843


In [ ]:
# Filtra expectativas longo prazo ~1 ano à frente e agrega pela média
df_tratado_bcb_odata_selic = (
    df_bruto_bcb_odata[2]
    .assign(
        DataReferencia = lambda x: pd.to_datetime(x.DataReferencia, format = "%Y"),
        data = lambda x: x.Data.dt.to_period("M").dt.to_timestamp(),
        horizonte = lambda x: ((x.DataReferencia - x.Data) / np.timedelta64(365, "D")).astype(int)
        )
    .query("horizonte == 1")
    .groupby(["data"], as_index = False)["expec_selic"]
    .mean()
)
df_tratado_bcb_odata_selic

,data,expec_selic
0,2000-08-01,11.947368
1,2000-09-01,12.000000
2,2000-10-01,12.000000
3,2000-11-01,12.153000
4,2000-12-01,12.537500
...,...,...
284,2024-04-01,8.528409
285,2024-05-01,8.935714
286,2024-06-01,9.000000
287,2024-07-01,9.000000


In [ ]:
# Filtra expectativas curto prazo ~1 mês à frente e agrega pela média
df_tratado_bcb_odata_cambio = (
    df_bruto_bcb_odata[3]
    .assign(
        DataReferencia = lambda x: pd.to_datetime(x.DataReferencia, format = "%m/%Y"),
        horizonte = lambda x: ((x.DataReferencia - x.Data) / np.timedelta64(30, "D")).astype(int),
        data = lambda x: x.Data.dt.to_period("M").dt.to_timestamp()
        )
    .query("horizonte == 1")
    .groupby(["data"], as_index = False)["expec_cambio"]
    .mean()
)
df_tratado_bcb_odata_cambio

,data,expec_cambio
0,2001-11-01,2.678333
1,2001-12-01,2.516316
2,2002-01-01,2.423636
3,2002-02-01,2.450000
4,2002-03-01,2.444000
...,...,...
269,2024-04-01,5.018810
270,2024-05-01,5.077281
271,2024-06-01,5.174210
272,2024-07-01,5.342414


In [ ]:
# Filtra expectativas curto prazo ~12 meses à frente e agrega pela média
df_tratado_bcb_odata_ipca_lp = (
    df_bruto_bcb_odata[4]
    .assign(data = lambda x: x.Data.dt.to_period("M").dt.to_timestamp())
    .groupby(["data"], as_index = False)["expec_ipca_12m"]
    .mean()
)
df_tratado_bcb_odata_ipca_lp

,data,expec_ipca_12m
0,2001-12-01,5.030000
1,2002-01-01,4.733636
2,2002-02-01,4.644737
3,2002-03-01,4.721500
4,2002-04-01,4.790000
...,...,...
268,2024-04-01,3.530673
269,2024-05-01,3.614862
270,2024-06-01,3.597295
271,2024-07-01,3.670978


In [ ]:
# Filtra expectativas médio prazo ~9 meses à frente e agrega pela média
df_tratado_bcb_odata_pib = (
    df_bruto_bcb_odata[5]
    .assign(
        DataReferencia = lambda x: pd.PeriodIndex(
            x.DataReferencia.str.replace(r"(\d{1})/(\d{4})", r"\2-Q\1", regex = True),
            freq = "Q"
            ).to_timestamp(),
        data = lambda x: x.Data.dt.to_period("Q").dt.to_timestamp(),
        horizonte = lambda x: ((x.DataReferencia - x.Data) / np.timedelta64(30, "D")).astype(int)
      )
    .query("horizonte == 9")
    .groupby(["data"], as_index = False)["expec_pib"]
    .mean()
)
df_tratado_bcb_odata_pib

,data,expec_pib
0,2001-10-01,3.891765
1,2002-01-01,4.119444
2,2002-04-01,3.736818
3,2002-07-01,3.140870
4,2002-10-01,2.438571
...,...,...
87,2023-07-01,1.453581
88,2023-10-01,1.926260
89,2024-01-01,2.000247
90,2024-04-01,1.989545


In [ ]:
# Filtra expectativas longo prazo ~1 ano à frente e agrega pela média
df_tratado_bcb_odata_primario = (
    df_bruto_bcb_odata[6]
    .assign(
        DataReferencia = lambda x: pd.to_datetime(x.DataReferencia, format = "%Y"),
        data = lambda x: x.Data.dt.to_period("M").dt.to_timestamp(),
        horizonte = lambda x: ((x.DataReferencia - x.Data) / np.timedelta64(365, "D")).astype(int)
        )
    .query("horizonte == 1")
    .groupby(["data"], as_index = False)["expec_primario"]
    .mean()
)
df_tratado_bcb_odata_primario

,data,expec_primario
0,2000-08-01,2.700000
1,2000-09-01,2.700000
2,2000-10-01,2.700000
3,2000-11-01,2.700000
4,2000-12-01,2.700000
...,...,...
284,2024-04-01,-0.481818
285,2024-05-01,-0.500000
286,2024-06-01,-0.502525
287,2024-07-01,-0.550678


In [ ]:
# Cruza dados de mesma frequência
df_tratado_bcb_odata_lista = [
    df_tratado_bcb_odata_ipca_mp,
    df_tratado_bcb_odata_ipca_lp,
    df_tratado_bcb_odata_selic,
    df_tratado_bcb_odata_cambio,
    df_tratado_bcb_odata_primario
  ]

df_tratado_bcb_odata_mensal = df_tratado_bcb_odata_ipca_cp.set_index("data")

for df in df_tratado_bcb_odata_lista:
  df_tratado_bcb_odata_mensal = df_tratado_bcb_odata_mensal.join(
      other = df.set_index("data"),
      how = "outer"
      )

df_tratado_bcb_odata_mensal

,expec_ipca_top5_curto_prazo,expec_ipca_top5_medio_prazo,expec_ipca_12m,expec_selic,expec_cambio,expec_primario
data,,,,,,
2000-08-01,NaN,NaN,NaN,11.947368,NaN,2.700000
2000-09-01,NaN,NaN,NaN,12.000000,NaN,2.700000
2000-10-01,NaN,NaN,NaN,12.000000,NaN,2.700000
2000-11-01,NaN,NaN,NaN,12.153000,NaN,2.700000
2000-12-01,NaN,NaN,NaN,12.537500,NaN,2.700000
...,...,...,...,...,...,...
2024-04-01,0.162133,0.276171,3.530673,8.528409,5.018810,-0.481818
2024-05-01,0.134762,0.500638,3.614862,8.935714,5.077281,-0.500000
2024-06-01,0.062500,0.454568,3.597295,9.000000,5.174210,-0.502525


## IPEADATA

In [ ]:
# Cruza dados do IPEADATA
df_tratado_ipeadata = df_bruto_ipeadata.copy()

for f in df_tratado_ipeadata.items():
  df_temp = f[1][0].assign(data = lambda x: pd.to_datetime(x.data, utc = True)).set_index("data")
  for df in f[1][1:]:
    df_temp = df_temp.join(
        other = df.assign(data = lambda x: pd.to_datetime(x.data, utc = True)).set_index("data"),
        how = "outer"
        )
  df_tratado_ipeadata[f[0]] = df_temp

df_tratado_ipeadata["Mensal"]

,icms,horas_trab_ind,papelao_ondulado,cons_papel,cons_gasolina_anp,emplacamento_veic,cons_energia_eletrica,ir,saldo_caged_antigo,saldo_caged_novo,ibov,swaps_di_360,cotacao_petroleo_fmi,indice_algueis_sp,exp_agro,exp_bens_duraveis
data,,,,,,,,,,,,,,,,
1957-01-01 02:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.920000,NaN,NaN,NaN
1957-02-01 02:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.920000,NaN,NaN,NaN
1957-03-01 03:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.920000,NaN,NaN,NaN
1957-04-01 03:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.920000,NaN,NaN,NaN
1957-05-01 03:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.920000,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-04-01 03:00:00+00:00,NaN,95.5,351026.0,79.043064,561.0,220807.0,47351.0,73726.087317,NaN,240033.0,-1.70,10.162273,88.397121,206.333663,8032.9938,512.8
2024-05-01 03:00:00+00:00,NaN,94.5,353015.0,NaN,540.0,194237.0,47050.0,71695.276420,NaN,131811.0,-3.04,10.400952,82.559130,208.980914,7488.9053,443.0
2024-06-01 03:00:00+00:00,NaN,94.0,341991.0,NaN,NaN,214289.0,45556.0,NaN,NaN,201705.0,1.48,10.889000,NaN,211.436889,NaN,NaN


In [ ]:
# Agrega dados de frequência diária para mensal por média
df_tratado_ipeadata["Mensal"] = (
    df_tratado_ipeadata["Mensal"]
    .reset_index()
    .assign(data = lambda x: x.data.dt.to_period("M").dt.to_timestamp())
    .set_index("data")
    .join(
        other = (
            df_tratado_ipeadata["Diária"]
            .reset_index()
            .assign(data = lambda x: x.data.dt.to_period("M").dt.to_timestamp())
            .set_index("data")
            .resample("MS")
            .mean()
        ),
        how = "outer"
      )
    .query("index >= '2000-01-01'")
)
df_tratado_ipeadata["Mensal"]

<ipython-input-42-41c12d153310>:5: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  .assign(data = lambda x: x.data.dt.to_period("M").dt.to_timestamp())
<ipython-input-42-41c12d153310>:11: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  .assign(data = lambda x: x.data.dt.to_period("M").dt.to_timestamp())


,icms,horas_trab_ind,papelao_ondulado,cons_papel,cons_gasolina_anp,emplacamento_veic,cons_energia_eletrica,ir,saldo_caged_antigo,saldo_caged_novo,ibov,swaps_di_360,cotacao_petroleo_fmi,indice_algueis_sp,exp_agro,exp_bens_duraveis,embi_br,ipc_s
data,,,,,,,,,,,,,,,,,,
2000-01-01,6529378.0,82.773956,188855.098811,77.887395,279.0,4600.0,25060.0,5043.680936,31198.0,NaN,-4.113276,21.240000,25.405714,NaN,174.8209,121.8,693.250000,NaN
2000-02-01,5956078.0,87.309235,195638.389128,75.435320,324.0,98936.0,25057.0,4120.602582,88175.0,NaN,7.761777,20.150000,27.370952,NaN,195.6361,194.3,704.050000,NaN
2000-03-01,5834411.0,90.177202,213088.275400,82.779406,278.0,92716.0,25662.0,5606.185192,39947.0,NaN,0.906002,18.950000,27.494493,NaN,260.5752,231.2,652.869565,NaN
2000-04-01,6320139.0,87.192720,206125.063586,73.932091,297.0,113309.0,25598.0,4634.431697,125071.0,NaN,-12.811448,19.330000,23.767000,NaN,449.8774,223.8,749.400000,NaN
2000-05-01,6412163.0,92.602816,215740.868021,79.859408,288.0,123089.0,25448.0,4047.302075,162837.0,NaN,-3.739461,21.630000,27.424203,NaN,513.1936,275.7,808.636364,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-04-01,NaN,95.500000,351026.000000,79.043064,561.0,220807.0,47351.0,73726.087317,NaN,240033.0,-1.700000,10.162273,88.397121,206.333663,8032.9938,512.8,212.227273,0.3025
2024-05-01,NaN,94.500000,353015.000000,NaN,540.0,194237.0,47050.0,71695.276420,NaN,131811.0,-3.040000,10.400952,82.559130,208.980914,7488.9053,443.0,211.272727,0.4800
2024-06-01,NaN,94.000000,341991.000000,NaN,NaN,214289.0,45556.0,NaN,NaN,201705.0,1.480000,10.889000,NaN,211.436889,NaN,NaN,228.684211,0.4700


## IBGE/SIDRA

In [ ]:
# Cruza dados do IBGE/SIDRA
df_tratado_ibge_sidra = df_bruto_ibge_sidra.copy()

for f in df_tratado_ibge_sidra.items():
  df_temp = (
      f[1][0]
      .iloc[1:]
      .assign(
          data = lambda x: pd.PeriodIndex(
            x.data.str.replace(r"(\d{4})(\d{1})(\d{1})", r"\1-\2\3" if f[0] == "Mensal" else r"\1-Q\3", regex = True),
            freq = "M" if f[0] == "Mensal" else "Q"
            ).to_timestamp()
        )
      .set_index("data")
  )
  for df in f[1][1:]:
    df_temp = df_temp.join(
        other = (
            df
            .iloc[1:]
            .assign(
                data = lambda x: pd.PeriodIndex(
                  x.data.str.replace(r"(\d{4})(\d{1})(\d{1})", r"\1-\2\3" if f[0] == "Mensal" else r"\1-Q\3", regex = True),
                  freq = "M" if f[0] == "Mensal" else "Q"
                  ).to_timestamp()
              )
            .set_index("data")
        ),
        how = "outer"
        )
  df_tratado_ibge_sidra[f[0]] = df_temp

df_tratado_ibge_sidra["Mensal"]

,prod_ind_papel,prod_ind_cimento,prod_ind_borracha,pms_volume,pmc_volume,pmc_volume_ampliado,prod_ind_transformacao,prod_ind_bens_consumo,prod_ind_embalagens_papel,prod_ind_metalurgia,prod_ind_geral,pop_ocupada,tx_desemprego,ipp_ind_geral,inpc,ipca,ipca_15
data,,,,,,,,,,,,,,,,,
1979-05-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.76,NaN,NaN
1979-06-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.00,NaN,NaN
1979-07-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.36,NaN,NaN
1979-08-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.79,NaN,NaN
1979-09-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.61,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-01,101.47416,3.0,105.45253,0.6,0.3,0.5,101.69927,0.6,2.6,95.47617,1.0,100203.0,7.9,0.35,0.19,0.16,0.36
2024-04-01,102.78457,6.3,104.14171,0.3,0.9,-0.8,102.05667,0.5,11.0,96.93427,-0.3,100804.0,7.5,0.67,0.37,0.38,0.21
2024-05-01,102.98731,3.0,104.54313,0.0,1.2,0.8,99.46322,-2.1,2.9,94.68485,-1.5,101331.0,7.1,0.36,0.46,0.46,0.44


## FRED

In [ ]:
# Cruza dados do FRED
df_tratado_fred = df_bruto_fred.copy()

for f in df_tratado_fred.items():
  df_temp = f[1][0].assign(observation_date = lambda x: pd.to_datetime(x.observation_date)).set_index("observation_date")
  for df in f[1][1:]:
    df_temp = df_temp.join(
        other = df.assign(observation_date = lambda x: pd.to_datetime(x.observation_date)).set_index("observation_date"),
        how = "outer"
        )
  df_temp = df_temp.rename_axis(index='data')
  df_tratado_fred[f[0]] = df_temp

,us_gov_sec_3m,fed_funds,us_gov_sec_1y,us_gov_sec_10y,us_gov_sec_2y,us_gov_sec_5y
data,,,,,,
1953-04-01,NaN,NaN,2.36,2.83,NaN,2.62
1953-05-01,NaN,NaN,2.48,3.05,NaN,2.87
1953-06-01,NaN,NaN,2.45,3.11,NaN,2.94
1953-07-01,NaN,NaN,2.38,2.93,NaN,2.75
1953-08-01,NaN,NaN,2.28,2.95,NaN,2.80
...,...,...,...,...,...,...
2024-03-01,5.47,5.33,4.99,4.21,4.59,4.20
2024-04-01,5.44,5.33,5.14,4.54,4.87,4.56
2024-05-01,5.46,5.33,5.16,4.48,4.86,4.50


In [ ]:
# Agrega dados de frequência diária para mensal por média
df_tratado_fred["Mensal"] = (
    df_tratado_fred["Mensal"]
    .set_index(pd.to_datetime(df_tratado_fred["Mensal"].index))
    .join(
        other = (
            df_tratado_fred["Diária"]
            .set_index(pd.to_datetime(df_tratado_fred["Diária"].index))
            .resample("MS")
            .mean()
        ),
        how = "outer"
      )
    .query("index >= '2000-01-01'")
)

## IFI

In [ ]:
# Representa em porcentagem dados do IFI
df_tratado_ifi = (
    df_bruto_ifi
    .assign(hiato_produto = lambda x: x.hiato_produto.mul(100))
    .query("data >= '2000-01-01'")
    .drop(labels = ["lim_inf", "lim_sup"], axis = "columns")
    .set_index("data")
)
df_tratado_ifi

,hiato_produto
data,
2000-03-01,-1.99
2000-06-01,-1.16
2000-09-01,-0.41
2000-12-01,0.16
2001-03-01,0.01
...,...
2023-03-01,0.55
2023-06-01,0.92
2023-09-01,0.70


# **Disponibilização de dados**
---

In [ ]:
# Cria pasta dados se não existir
pasta = "dados"
if not os.path.exists(pasta):
  os.makedirs(pasta)

## BCB/SGS

In [ ]:
# Salva dados como arquivo .parquet
for df in df_tratado_bcb_sgs.items():
  df[1].to_parquet(f"{pasta}/df_bcb_sgs_{df[0]}.parquet")

NameError: name 'df_tratado_bcb_sgs' is not defined

## BCB/ODATA

In [ ]:
# Salva dados como arquivo .parquet
df_tratado_bcb_odata_mensal.to_parquet(f"{pasta}/df_bcb_odata_mensal.parquet")
df_tratado_bcb_odata_pib.set_index("data").to_parquet(f"{pasta}/df_bcb_odata_trimestral.parquet")

## IPEADATA

In [ ]:
# Salva dados como arquivo .parquet
for df in df_tratado_ipeadata.items():
  df[1].to_parquet(f"{pasta}/df_ipeadata_{df[0]}.parquet")

## IBGE/SIDRA

In [ ]:
# Salva dados como arquivo .parquet
for df in df_tratado_ibge_sidra.items():
  df[1].query("index >= '2000-01-01'").to_parquet(f"{pasta}/df_ibge_sidra_{df[0]}.parquet")

## FRED

In [ ]:
# Salva dados como arquivo .parquet
for df in df_tratado_fred.items():
  df[1].query("index >= '2000-01-01'").to_parquet(f"{pasta}/df_fred_{df[0]}.parquet")

## IFI

In [ ]:
# Salva dados como arquivo .parquet
df_tratado_ifi.to_parquet(f"{pasta}/df_ifi_trimestral.parquet")

## Tabelas por frequência

In [ ]:
# Diária
df_diaria = (
    df_tratado_bcb_sgs["Diária"]
    .join(
        other=df_tratado_ipeadata["Diária"].reset_index().assign(
            data=lambda x: pd.to_datetime(x['data'].dt.strftime("%Y-%m-%d"))
        ).set_index("data"),
        how="outer"
    )
    .join(other=df_tratado_fred["Diária"], how="outer")
    .reset_index()
    .assign(data=lambda x: pd.to_datetime(x['data']))
    .query("data >= @pd.to_datetime('2000-01-01')")
    .set_index('data')
)
df_diaria.to_parquet(f"{pasta}/df_diaria.parquet")

In [ ]:
# Mensal
temp_lista = [
    df_tratado_bcb_sgs["Mensal"],
    df_tratado_bcb_odata_mensal,
    df_tratado_ipeadata["Mensal"],
    df_tratado_ibge_sidra["Mensal"],
    df_tratado_fred["Mensal"]
]

df_mensal = (
  temp_lista[0]
  .join(other = temp_lista[1:], how = "outer")
  .query("index >= @pd.to_datetime('2000-01-01')")
  .astype(float)
  )
df_mensal.to_parquet(f"{pasta}/df_mensal.parquet")

In [ ]:
# Trimestral
temp_lista = [
    df_tratado_bcb_sgs["Trimestral"],
    df_tratado_bcb_odata_pib.set_index("data"),
    df_tratado_ibge_sidra["Trimestral"],
    df_tratado_fred["Trimestral"],
    df_tratado_ifi
]

df_trimestral = (
  temp_lista[0]
  .join(other = temp_lista[1:], how = "outer")
  .query("index >= @pd.to_datetime('2000-01-01')")
  .astype(float)
)
df_trimestral.index = pd.to_datetime(df_trimestral.index)
df_trimestral.to_parquet(f"{pasta}/df_trimestral.parquet")

In [ ]:
# Anual
df_anual = (
  df_tratado_bcb_sgs["Anual"]
  .query("index >= @pd.to_datetime('2000-01-01')")
  .astype(float)
)
df_anual.to_parquet(f"{pasta}/df_anual.parquet")


,meta_inflacao
data,
2000-01-01,6.00
2001-01-01,4.00
2002-01-01,3.50
2003-01-01,4.00
2004-01-01,5.50
2005-01-01,4.50
2006-01-01,4.50
2007-01-01,4.50
2008-01-01,4.50
